# Upload images from server filesystem to Girder using python API

## From: Frink /HCP_Data/HCP_BedpostData/
## To: Candygram 
http://candygram.neurology.emory.edu:8080/#collection/5c48db6de62914004de6350f/folder/5c743889e62914004d6f974c

In [ ]:
# Ensure girder_client installed:
! pip install girder_client

In [1]:
import girder_client
import sys,os,glob

In [2]:
# Subject Data location:
subjRootDir = "/data/HCP_BedpostData/"

# Source subject nodiff:
diffData = "/T1w/Diffusion/nodif*"

# Source ROIs
dtiSpaceMasks = "addlInfo/%s/DTI_ROIs/*"

In [3]:
# Admin login:
cbGirder = girder_client.GirderClient(apiUrl="http://candygram.neurology.emory.edu:8080/api/v1")
cbGirder.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [4]:
# Destination of Subject data in DTI Space :
subjDTIPath = "/collection/FG/HCP_TractographyData/SubjectRegistrationResults/DTI_Space"
subjDtiTarget = cbGirder.get("resource/lookup?path=" + subjDTIPath)

In [5]:
# List of Subjects:
subjList = [x for x in os.listdir(subjRootDir) if 'addlInfo' not in x]
print(len(subjList),"Subjects have Bedpost Data")

1009 Subjects have Bedpost Data


## Upload Source nodiff images images to Destination
Source: Server filesystem <br>
Destination: Girder

In [ ]:
# Iterate through subjects:
for s in subjList: 
    # Create destination folder (or get existing):
    grdrTgtSubjFolder = cbGirder.createFolder(subjDtiTarget['_id'],
                                              name=s,
                                              description="Subject %s DTI Space Images" % s,
                                              reuseExisting=True)
    
    ## Items in subject folder (Destination):
    itemsInSubjFolder = list(cbGirder.listItem(grdrTgtSubjFolder['_id']))
    
    # List of tuples for Destination images: (filename, file_size)
    # Used to detect if Source images already uploaded to Girder:
    curSubjFiles = [ (x['name'],x['size']) for  x  in itemsInSubjFolder  ]

    # List of DTI nodiff images on server (Source):
    dtiBaseFiles = glob.glob(subjRootDir+s+diffData)
    for f in dtiBaseFiles:
        # Check if file exists in Girder (Destination):
        if((os.path.basename(f),os.path.getsize(f) ) in curSubjFiles):
            print("Already uploaded: %s" % f)
        else:
            print("Uploading: %s" % f)
            # Upload Source to Destination:
            grdrItemInfo = cbGirder.uploadFileToFolder(grdrTgtSubjFolder['_id'],
                                                       f,
                                                       filename=os.path.basename(f))
            # Upload metadata:
            cbGirder.addMetadataToItem(grdrItemInfo['itemId'],
                                       {"XTK":{"type":"volume2d"}})
            print("Done: %s" % f)
           

# Upload ROI Masks:
MNI masks converted to subject DTI Space

In [ ]:
# Iterate through subjects:
for s in subjList:
    print('Subject: %s' % s)
    grdrTgtSubjFolder = cbGirder.createFolder(subjDtiTarget['_id'],
                                              name=s,
                                              description="Subject %s DTI Space Images" % s,
                                              reuseExisting=True)

    grdrDTIROISubjFolder = cbGirder.createFolder(grdrTgtSubjFolder['_id'],
                                                 name='DTI_ROIs',
                                                 description="Subject %s DTI Space ROIs" % s,
                                                 reuseExisting=True)
    
    ## Items in subject ROI folder (Destination):
    itemsInROIFolder = list(cbGirder.listItem(grdrDTIROISubjFolder['_id']))
    
    # List of tuples for Destination images: (filename, file_size)
    # Used to detect if Source images already uploaded to Girder:
    curSubjFiles = [ (x['name'],x['size']) for  x  in itemsInROIFolder  ]
    
    # List of DTI ROI Mask images on server (Source):
    # Ex: '/data/HCP_BedpostData/addlInfo/379657/DTI_ROIs/*'
    dtiBaseFiles = glob.glob(subjRootDir+dtiSpaceMasks % s)
    for f in dtiBaseFiles:
        # Check if file exists in Girder (Destination):
        if((os.path.basename(f),os.path.getsize(f) ) in curSubjFiles):
            print("Already Uploaded: %s" % f)
        else:
            print("Uploading: %s" % f)
            # Upload Source to Destination:
            grdrItemInfo = cbGirder.uploadFileToFolder(grdrDTIROISubjFolder['_id'],
                                                       f,
                                                       filename=os.path.basename(f))

            # Upload metadata:
            cbGirder.addMetadataToItem(grdrItemInfo['itemId'],
                                       {"XTK":{"type":"volume2d"}})
            print("Done: %s" % f)

# Upload Specification file in each subject folder:
### Spec defines data visualized parameters and load order

In [11]:
s = subjList[0]

grdrTgtSubjFolder = list(cbGirder.listFolder(subjDtiTarget['_id'],
                                                 name=s))

itemsInSubjFolder = list(cbGirder.listItem(grdrTgtSubjFolder[0]['_id']))



In [21]:
foldersInSubjFolder

[{'_accessLevel': 2,
  '_id': '5c86d93be62914004d778cc1',
  '_modelType': 'folder',
  'baseParentId': '5c48db6de62914004de6350f',
  'baseParentType': 'collection',
  'created': '2019-03-11T21:55:07.728000+00:00',
  'creatorId': '5956b83b92ca9a000d9c0488',
  'description': 'Subject 379657 DTI Space ROIs',
  'name': 'DTI_ROIs',
  'parentCollection': 'folder',
  'parentId': '5c86d6a4e62914004d776521',
  'public': True,
  'size': 271280,
  'updated': '2019-03-11T21:55:07.728000+00:00'}]

In [ ]:
# DTI base data: first: nodif, second: nodif_brain, third: nodif_brain_mask
dtiFilesForPapaya = ['nodif.nii.gz',
                     'nodif_brain.nii.gz',
                     'nodif_brain_mask.nii.gz']

# Iterate through subjects:
for s in subjList[0:1]:
    papayaSpecDataDict = {}
    
    # Subject folder:
    grdrTgtSubjFolder = list(cbGirder.listFolder(subjDtiTarget['_id'],
                                                 name=s))
    # Images in subject folder:
    itemsInSubjFolder = list(cbGirder.listItem(grdrTgtSubjFolder[0]['_id']))
    
    # Subject subfolders: Ex. 'DTI_ROIs'
    foldersInSubjFolder = list(cbGirder.listFolder(grdrTgtSubjFolder[0]['_id']) )
    
    # Iterate through itemsInSubjFolder:
    for i in itemsInSubjFolder:
        # if image in dtiFilesForPapaya, create Specification entry:
        if i['name'] in dtiFilesForPapaya:
            # file ID:
            itemFiles = cbGirder.listFile(i['_id'])
            
            # An item can have one or more files:
            grdrFileId = None
            for itf in itemFiles:
                if itf['name'] == i['name']:
                    grdrFileId = itf['_id']
                    break
            
            # Specification Entry:
            papayaSpecDataDict[i['name']] = {'displayName': i['name'].replace('nii.gz',''), 
                                             'grdrFileId': grdrFileId,
                                             "alpha": 1, 
                                             "visible": 1}
            
    # Order Specification files:
    for d in dtiFilesForPapaya:
        if d in papayaSpecDataDict:
            papayaSpec.append( papayaSpecDataDict[d]  )
    # Upload metadata to Subject folder:        
    cbGirder.addMetadataToFolder(grdrTgtSubjFolder[0]['_id'], {"papayaSpec":papayaSpec})
    